In [ ]:
#A*x = 0
using LinearAlgebra
struct HRep{T}
   n :: Int64 #input size
   A :: AbstractArray{T}
end
struct VRep{T}
   n :: Int64 #input size
   A :: AbstractArray{T}
end

function vrep(x::HRep) 
    VRep(x.n, nullspace(x.A))
end
function hrep(x::VRep)
    HRep(x.n, nullspace(x.A')')
end


function meet(x::HRep, y::HRep)
    @assert x.n == y.n
    HRep(x.n, [x.A ; y.A])
end
function meet(x::VRep, y::VRep)
    @assert x.n == y.n
    xh = hrep(x)
    yh = hrep(y)
    HRep(x.n, [xh.A ; yh.A])
end
function meet(x::HRep, y::VRep)
    @assert x.n == y.n
    yh = hrep(y)
    HRep(x.n, [x.A ; yh.A])
end
function meet(x::VRep, y::HRep)
    @assert x.n == y.n
    xh = hrep(x)
    HRep(x.n, [xh.A ; y.A])
end


function join(x::VRep, y::VRep)
    @assert x.n == y.n
    VRep(x.n, [x.A y.A])
end
function join(x::HRep, y::HRep)
    yv = vrep(y)
    join(x,yv)
end
function join(x::VRep, y::HRep)
        xv = vrep(x)
        join(xv,y)
end
function join(x::HRep, y::HRep)
        xv = vrep(x)
        join(xv,y)
end
    
function rid(n)
    VRep(n, [ Matrix(I,n,n) ; Matrix(I,n,n) ]  )
end


function rsub(r :: VRep,p :: HRep)
    all(isapprox.(p.A * r.A , 0; atol=eps(Float64), rtol=0))
end

function rsub(r :: VRep, p :: VRep)
    p = hrep(p)
    all(isapprox.(p.A  * r.A , 0; atol=eps(Float64), rtol=0))
end

function heq(p,q)
   rsub(p,q) && rsub(q,p) 
end


function converse(p :: VRep)
   n, g = size(p.A)
    VRep( n - p.n  , [p.A[p.n + 1 : end, :] ;  p.A[1 : p.n, :]  ]) 
end

function converse(p :: HRep)
   c, n = size(p.A)
   HRep( n - p.n  , [p.A[:, p.n + 1 : end]  p.A[:, 1 : p.n]]) 
end

function top(n)
   VRep(n, Matrix(I, n, n)) 
end 
function bottom(n)
    HRep(n, Matrix(I, n, n)) 
end


function compose(x::HRep, y::HRep)
    cx, nx = size(x.A)
    cy, ny = size(y.A)    
    na = x.n
    nb = nx - x.n # which should equal y.n
    nc = ny - y.n
    B = [                x.A          zeros( cx, ny - y.n) ; 
         zeros( cy, x.n  )      y.A                        ]
    C = nullspace(B)
    return VRep([  C[1 : n.x, :]          ; 
                   C[ nx+y.n + 1:end, :] ])
end

In [2]:
?Type

search: Type typeof TypeVar typemin typemax TypeError typejoin typeassert



No documentation found.

`Core.Type` is of type `UnionAll`.

# Summary

```
struct UnionAll <: Type{T}
```

# Fields

```
var  :: TypeVar
body :: Any
```

# Supertype Hierarchy

```
UnionAll <: Type{T} <: Any
```


In [3]:
TYPE

UndefVarError: UndefVarError: TYPE not defined

In [4]:
using Catlab

┌ Info: Precompiling Catlab [134e5e36-593f-5add-ad60-77f754baafbe]
└ @ Base loading.jl:1242


In [40]:
macroexpand(Main,quote  @signature MyCategory(Ob,Hom) begin
  Ob::TYPE
  Hom(dom::Ob, codom::Ob)::TYPE

  id(A::Ob)::Hom(A,A)
  compose(f::Hom(A,B), g::Hom(B,C))::Hom(A,C) <= (A::Ob, B::Ob, C::Ob)
            end end)

quote
    #= In[40]:1 =#
    begin
        eval(Catlab.GAT.signature_code(Catlab.GAT.Typeclass(:MyCategory, Symbol[:Ob, :Hom], Catlab.GAT.Signature(Catlab.GAT.TypeConstructor[Catlab.GAT.TypeConstructor(:Ob, Symbol[], OrderedCollections.OrderedDict{Symbol,Union{Expr, Symbol}}(), nothing), Catlab.GAT.TypeConstructor(:Hom, Symbol[:dom, :codom], OrderedCollections.OrderedDict{Symbol,Union{Expr, Symbol}}(:dom => :Ob,:codom => :Ob), nothing)], Catlab.GAT.TermConstructor[Catlab.GAT.TermConstructor(:id, Symbol[:A], :(Hom(A, A)), OrderedCollections.OrderedDict{Symbol,Union{Expr, Symbol}}(:A => :Ob), nothing), Catlab.GAT.TermConstructor(:compose, Symbol[:f, :g], :(Hom(A, C)), OrderedCollections.OrderedDict{Symbol,Union{Expr, Symbol}}(:A => :Ob,:B => :Ob,:C => :Ob,:f => :(Hom(A, B)),:g => :(Hom(B, C))), nothing)]), Catlab.Meta.JuliaFunction[]), nothing, Any[]))
        begin
            $(Expr(:meta, :doc))
            MyCategory
        end
    end
end

In [19]:
struct MyBoy
end
macroexpand( Main, quote
@instance MyCategory(MyBoy, Int64) begin
    dom(x) = MyBoy()
    codom(x) = MyBoy()
    id(mahboy) = 0
    compose(x,y) = x = y
end
        end)

quote
    #= In[19]:4 =#
    begin
        eval(Catlab.GAT.instance_code(MyCategory, Symbol[:MyBoy, :Int64], Catlab.Meta.JuliaFunction[Catlab.Meta.JuliaFunction(:(dom(x)), nothing, quote
    #= In[19]:5 =#
    MyBoy()
end, nothing), Catlab.Meta.JuliaFunction(:(codom(x)), nothing, quote
    #= In[19]:6 =#
    MyBoy()
end, nothing), Catlab.Meta.JuliaFunction(:(id(mahboy)), nothing, quote
    #= In[19]:7 =#
    0
end, nothing), Catlab.Meta.JuliaFunction(:(compose(x, y)), nothing, quote
    #= In[19]:8 =#
    x = y
end, nothing)]))
        begin
            $(Expr(:meta, :doc))
            abstract type ##instance_doc#373 end
        end
    end
end

In [17]:
typeof(MyBoy())

MyBoy

In [21]:
using Polyhedra

┌ Info: Precompiling Polyhedra [67491407-f73d-577b-9b50-8179a7c68029]
└ @ Base loading.jl:1242


In [22]:
conichull([1,0],[0,1])

V-representation Polyhedra.RaysHull{Int64,Array{Int64,1},Int64}:
2-element iterator of Ray{Int64,Array{Int64,1}}:
 Ray([1, 0])
 Ray([0, 1])

In [23]:
conichull(Line([1, 0]), Line([0, 1])) # full space

V-representation LinesHull{Int64,Array{Int64,1},Int64}:
2-element iterator of Line{Int64,Array{Int64,1}}:
 Line([1, 0])
 Line([0, 1])

In [34]:
[ Line(convert(Array,col)) for col in  eachcol([1 0 ; 0 1])]

2-element Array{Line{Int64,Array{Int64,1}},1}:
 Line([1, 0])
 Line([0, 1])

In [36]:
using LinearAlgebra

In [39]:
n = 3
vrep([ Line(convert(Array,col)) for col in  eachcol([ Matrix(I, n,n) ; Matrix(I,n,n)])  ])

V-representation LinesHull{Bool,Array{Bool,1},Int64}:
3-element iterator of Line{Bool,Array{Bool,1}}:
 Line(Bool[1, 0, 0, 1, 0, 0])
 Line(Bool[0, 1, 0, 0, 1, 0])
 Line(Bool[0, 0, 1, 0, 0, 1])

In [33]:
[1 2]

1×2 Array{Int64,2}:
 1  2

In [32]:
?Line

search: Line lines linetype LinesHull linespace LinearIndices LineNumberNode



```
struct Line{T, AT <: AbstractVector{T}}
    a::AT
end
```

The conic hull of `a` and `-a`, i.e. the set of points `λa` where `λ` is any real number.


In [42]:
?∘

"∘" can be typed by \circ<tab>

search: ∘



```
f ∘ g
```

Compose functions: i.e. `(f ∘ g)(args...)` means `f(g(args...))`. The `∘` symbol can be entered in the Julia REPL (and most editors, appropriately configured) by typing `\circ<tab>`.

# Examples

```jldoctest
julia> map(uppercase∘first, ["apple", "banana", "carrot"])
3-element Array{Char,1}:
 'A'
 'B'
 'C'
```


In [47]:
Iterator(Bool)

UndefVarError: UndefVarError: Iterator not defined

In [50]:
?Bool

search: Bool BoundsError BlockElimination AbstractPolyhedraOptimizer



```
Bool <: Integer
```

Boolean type, containing the values `true` and `false`.


In [51]:
?iterate

search: iterate InteractiveUtils isinteractive Iterators RepIterator



```
iterate(iter [, state]) -> Union{Nothing, Tuple{Any, Any}}
```

Advance the iterator to obtain the next element. If no elements remain, `nothing` should be returned. Otherwise, a 2-tuple of the next element and the new iteration state should be returned.

---

```
iterate(s::AbstractString, i::Integer) -> Union{Tuple{<:AbstractChar, Int}, Nothing}
```

Return a tuple of the character in `s` at index `i` with the index of the start of the following character in `s`. This is the key method that allows strings to be iterated, yielding a sequences of characters. If `i` is out of bounds in `s` then a bounds error is raised. The `iterate` function, as part of the iteration protocol may assume that `i` is the start of a character in `s`.

See also: [`getindex`](@ref), [`checkbounds`](@ref)


In [55]:
Base.IteratorSize(Bool)

Base.HasShape{0}()

In [56]:
f(x::Bool) = 7
f(x::Integer) = 8

f (generic function with 2 methods)

In [57]:
f(true)

7

In [59]:
typemin(Bool)

false

In [60]:
typemax(Bool)

true

In [61]:
next(false)

UndefVarError: UndefVarError: next not defined

In [66]:
[print(x) for x in false:true]

falsetrue

2-element Array{Nothing,1}:
 nothing
 nothing

In [67]:
typemin(Tuple{Bool,Bool})

MethodError: MethodError: no method matching typemin(::Type{Tuple{Bool,Bool}})
Closest candidates are:
  typemin(!Matched::Type{Bool}) at bool.jl:6
  typemin(!Matched::Type{Int8}) at int.jl:665
  typemin(!Matched::Type{UInt8}) at int.jl:667
  ...

In [68]:
Tuple{Bool,Bool}

Tuple{Bool,Bool}

In [69]:
size(Bool)

MethodError: MethodError: no method matching size(::Type{Bool})
Closest candidates are:
  size(!Matched::BitArray{1}) at bitarray.jl:77
  size(!Matched::BitArray{1}, !Matched::Integer) at bitarray.jl:81
  size(!Matched::Core.Compiler.StmtRange) at show.jl:1585
  ...

In [70]:
card(Bool)

UndefVarError: UndefVarError: card not defined

In [71]:
[(:Bool, 2)]

1-element Array{Tuple{Symbol,Int64},1}:
 (:Bool, 2)

In [72]:
(:Bool,:Bool)

(:Bool, :Bool)

In [73]:
struct Test
    x :: Type
end

Test(Tuple{Bool,Bool})

Test(Tuple{Bool,Bool})

In [ ]:
function card(x)
    if x == Tuple{}

end

In [ ]:
struct HRep{n,m,T}
    A :: AbstractArray{T}
end

In [74]:
AbstractArray{Float64, 3}

AbstractArray{Float64,3}

In [78]:
[1  ; 2  ; 3]

3-element Array{Int64,1}:
 1
 2
 3

In [79]:
struct Val{T}
end

Val(x) = Val{x}()

Val

In [80]:
Val(true)

Val{true}()

In [81]:
card(::Bool) = 2


card (generic function with 1 method)

In [82]:
card(true)

2

In [83]:
card(::Tuple{T,S}) = card(typemin(T), typemin(S))

UndefVarError: UndefVarError: T not defined

In [90]:
#I = (:I, 1)
#V = (:V, 1)
#IV = (I,V)
struct PrimTyp
    sym :: Symbol
    card :: Int64
end

In [108]:
card( x::Tuple ) = card(x[1]) + card(x[2])
card(x :: PrimType) = x.card
card(x :: PrimTyp) = x.card

card (generic function with 4 methods)

In [111]:
struct VRep2{T,Dom,Cod}
    dom :: Dom
    cod :: Cod
    data :: AbstractArray{T}
end

In [95]:
i = PrimTyp(:I, 1)

PrimTyp(:I, 1)

In [110]:
card(((i,i), i))

3

In [100]:
typeof((i,i))

Tuple{PrimTyp,PrimTyp}

In [105]:
(10,4)

LoadError: syntax: extra token "0.0" after end of expression